### <span style="color:red">IMPORTANT: Only modify cells which have the following comment:</span>
```python
# Modify this cell
```
##### <span style="color:red">Do not add any new cells when you submit the homework</span>

# Setting Up Notebook

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

In [15]:
import numpy as np
import math
from numpy import linalg as LA
import pickle

# Exercise:
The function **computeCov** computes the covariance matrix using RDDs. The code allows undefined entries and calculates the covariance without bias. 

Your homework is to complete the missing parts in **computeCov** (Marked with `...`) so that it calculates the covariance correctly.

    Note: The functions and libraries in the cell below will be useful to you

In [6]:
def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

In [7]:
np.ones((5,))

array([ 1.,  1.,  1.,  1.,  1.])

In [10]:
# Modify this cell

def computeCov(RDDin):
    # input: RDDin is an RDD of np arrays, all of the same length
    
    # we insert 1 at the beginning of each vector so the calculation also yields the mean vector
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
    print len(RDD)
    # separating map and reduce does not matter, since Spark uses lazy execution.
    OuterRDD = RDD.map(lambda x: outerProduct(x)) #...    #<-- do mapping here
    (S,N)=OuterRDD.reduce(lambda x,y: sumWithNan(x,y))  # ...  #<-- do reducing here
    
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    print 'shape of E=',E.shape,'shape of NE=',NE.shape
    Mean=E/NE
    O=S[1:,1:]
    NO=np.float64(N[1:,1:])
    
    Cov= (O/NO) - np.outer(Mean,Mean)  #...... # This is the covariance matrix
    
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}


In [17]:
import Tester.SmallPCA as pca
pca.exercise(computeCov, sc)

ValueError: insecure string pickle